In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import pandas as pd

In [5]:
df = pd.read_csv('full_devices.csv', encoding_errors='ignore')
print(df.shape)
df.head()

(124494, 12)


,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [22]:
df.groupby('failure').agg(num_unique_devices=('device','nunique'),
                          num_incidents=('device','count')
                          )

,num_unique_devices,num_incidents
failure,,
0,1169,124388
1,106,106


In [45]:
#df.device.value_counts()

#df[df['failure']==1][['device', 'failure']].drop_duplicates()
devices_with_failure = df[df['failure']==1].device.unique().tolist()
print(devices_with_failure)


devices_without_failure = df[~df['device'].isin(devices_with_failure)].device.unique().tolist()
print(devices_without_failure)

['S1F0RRB1', 'S1F0CTDN', 'W1F0PNA5', 'W1F13SRV', 'W1F1230J', 'W1F0T034', 'S1F0GG8X', 'S1F023H2', 'S1F0QY11', 'S1F0S2WJ', 'W1F0Z1W9', 'W1F15S4D', 'Z1F0LVPW', 'Z1F0NVZA', 'Z1F1FCH5', 'S1F0P3G2', 'W1F0F6BN', 'W1F0P114', 'W1F0X4FC', 'S1F0LCTV', 'W1F03DP4', 'W1F0FW0S', 'S1F10E6M', 'S1F11MB0', 'W1F0SGHR', 'W1F0VDH2', 'W1F0TA59', 'Z1F0LVGY', 'Z1F0MCCA', 'Z1F0P5D9', 'W1F0NZZZ', 'W1F0T074', 'S1F0DSTY', 'S1F0TQCV', 'Z1F04GCH', 'W1F08EDA', 'W1F1C9TE', 'S1F0S4CA', 'W1F19BPT', 'Z1F130LH', 'S1F0GJW3', 'S1F0LD2C', 'W1F0Q8FH', 'Z1F0FSBY', 'W1F0Z4EA', 'Z1F0QH0C', 'S1F0S4T6', 'W1F1CDDP', 'S1F0S57T', 'S1F0JD7P', 'S1F13H80', 'Z1F148T1', 'S1F0RSZP', 'S1F0GKFX', 'S1F0LCVC', 'W1F1BZTM', 'Z1F1RJFA', 'S1F13589', 'S1F136J0', 'S1F0F4EB', 'W1F1C9WG', 'S1F0RR35', 'Z1F1653X', 'Z1F1AG5N', 'W1F0KCP2', 'W1F0M35B', 'Z1F1901P', 'S1F0GKL6', 'Z1F0K451', 'W1F03D4L', 'W1F0FKWW', 'S1F0PJJW', 'W1F0X5GW', 'S1F0L0DW', 'W1F0WBTM', 'S1F0GSD9', 'S1F0QF3R', 'W1F0Z3KR', 'W1F0M4BZ', 'Z1F0B4XZ', 'W1F0GCAZ', 'Z1F0LSNZ', 'Z1F1VQFY', 'Z1

In [39]:
df[df['device'].isin(devices_with_failure)].sort_values(['device','date']).head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
7,2015-01-01,S1F023H2,0,141503600,0,0,1,19,494462,16,16,3
1170,2015-01-02,S1F023H2,0,161679800,0,0,1,19,495730,16,16,3
2333,2015-01-03,S1F023H2,0,182358672,0,0,1,19,496974,16,16,3
3496,2015-01-04,S1F023H2,0,204752808,0,0,1,19,497559,16,16,3
4658,2015-01-05,S1F023H2,0,226982888,0,0,1,19,498753,16,16,3


In [52]:
device = 'W1F0T034'

device = 'W1F1230J'

print(len(df[df['device']==device].sort_values(['device','date'])))
df[df['device']==device].sort_values(['device','date']).tail(20)

14


,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
815,2015-01-01,W1F1230J,0,160609528,0,0,0,14,307719,0,0,0
1978,2015-01-02,W1F1230J,0,187043960,0,0,0,14,309041,0,0,0
3141,2015-01-03,W1F1230J,0,212033256,0,0,0,14,310378,0,0,0
4303,2015-01-04,W1F1230J,0,231117752,0,0,0,14,311765,0,0,0
5464,2015-01-05,W1F1230J,0,14394264,0,0,0,14,313068,0,0,0
6567,2015-01-06,W1F1230J,0,43272744,0,0,0,14,314337,0,0,0
7419,2015-01-07,W1F1230J,0,61165160,0,0,0,14,315687,0,0,0
8175,2015-01-08,W1F1230J,0,81394392,0,0,0,14,317064,0,0,0
8931,2015-01-09,W1F1230J,0,110557480,0,0,0,14,318382,0,0,0
9686,2015-01-10,W1F1230J,0,134310728,0,0,0,14,319719,0,0,0


In [50]:
device='S1F01085'

print(len(df[df['device']==device].sort_values(['device','date'])))
df[df['device']==device].sort_values(['device','date']).tail(50)

6


,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1163,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7
2326,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7
3489,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7
4651,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7
5812,2015-01-06,S1F01085,0,128832128,56,0,52,6,409404,0,0,7
